In [1]:
import numpy as np
from copy import deepcopy
import pygame
from game import Game
import time
from GUI import *
import random
import math

pygame 2.1.2 (SDL 2.0.18, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Weight and bias dimensions for the NN
first_layer_hidden_weights = (32,90) #32,90
first_layer_hidden_bias = (1,90) #90
second_layer_hidden_weights = (90,40) #90,40 
second_layer_hidden_bias = (1,40) #40
third_layer_hidden_weights = (40,10) #40,10
third_layer_hidden_bias = (1,10) #10

class Evol_Player(object):
    def __init__(self, number, first_layer_weights, first_layer_bias, second_layer_weights, second_layer_bias, third_layer_weights, third_layer_bias):
        self.number = number
        self.score = 0
        self.first_layer_weights = first_layer_weights
        self.first_layer_bias = first_layer_bias
        self.second_layer_weights = second_layer_weights
        self.second_layer_bias = second_layer_bias
        self.third_layer_weights = third_layer_weights
        self.third_layer_bias = third_layer_bias

        self.win = 0
        self.loss = 0
        self.draw = 0

    def getWeights(self):
        return self.first_layer_weights, self.first_layer_bias,  self.second_layer_weights, self.second_layer_bias,  self.third_layer_weights,  self.third_layer_bias

# Used in the NN
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

def evolutionary_player(count):
    
    first_layer_weights = np.random.normal(0, scale=1.0, size=first_layer_hidden_weights)
    first_layer_bias = np.random.normal(0, scale=1.0, size=first_layer_hidden_bias)
    second_layer_weights = np.random.normal(0, scale=1.0, size=second_layer_hidden_weights)
    second_layer_bias = np.random.normal(0, scale=1.0, size=second_layer_hidden_bias)
    third_layer_weights = np.random.normal(0, scale=1.0, size=third_layer_hidden_weights)
    third_layer_bias = np.random.normal(0, scale=1.0, size=third_layer_hidden_bias)

    return Evol_Player(count, first_layer_weights, first_layer_bias, second_layer_weights, second_layer_bias, third_layer_weights, third_layer_bias)

# Use this Neural Network as the heuristic function for the minimax tree
def predict_nn(board, player):

    first_hidden_output = tanh(np.dot(board, player.first_layer_weights) + player.first_layer_bias)
    second_hidden_output = tanh(np.dot(first_hidden_output, player.second_layer_weights) + player.second_layer_bias)
    third_layer_output = tanh(np.dot(second_hidden_output, player.third_layer_weights) + player.third_layer_bias)

    output = np.sum(third_layer_output)
    
    return output

def createNeuralNetwork(offspring1AfterMutation, offspring2AfterMutation, count):

    first_layer_bias1, second_layer_bias1, third_layer_bias1, first_layer_weights1, second_layer_weights1, third_layer_weights1 = offspring1AfterMutation
    first_layer_bias2, second_layer_bias2, third_layer_bias2, first_layer_weights2, second_layer_weights2, third_layer_weights2 = offspring2AfterMutation

    player1 = Evol_Player(count + 1, first_layer_weights1, first_layer_bias1, second_layer_weights1, second_layer_bias1, third_layer_weights1, third_layer_bias1)
    player2 = Evol_Player(count + 2, first_layer_weights2, first_layer_bias2, second_layer_weights2, second_layer_bias2, third_layer_weights2, third_layer_bias2)

    return player1, player2


In [3]:
def simulate_move(piece, move, board, game, skip):
    """
    Simulates a move on the board by moving a piece, capturing any opponent's piece, and updating the game state.
    and return updated game board after the move.
    """
    board.move(piece, move[0], move[1])  # Move the piece on the board.
    if skip:
        board.remove(skip)  # Remove opponent's piece that was captured during the move.

    return board

In [4]:
def get_all_moves(board, color, game):
    """
    Gets all valid moves for a given color on the current game board.
    and returns a List of all possible game boards after making each move.
    """
    moves = []
    priority = board.priority_moves(color)
    # print(priority)
    if len(priority)>0:
        for piece in board.get_all_pieces(color):
            valid_moves, jump = board.get_valid_moves(piece)  # Get all valid moves for the piece.
            for i in valid_moves:
                # print(i)
                if i in priority:
                    for move, skip in valid_moves.items():
                        if board != None:
                            temp_board = deepcopy(board)
                            temp_piece = temp_board.get_piece(piece.row, piece.col)
                            new_board = simulate_move(temp_piece, move, temp_board, game, skip)  # Simulate the move on the board.
                            moves.append(new_board)
    else:
        for piece in board.get_all_pieces(color):
            valid_moves, jump = board.get_valid_moves(piece)  # Get all valid moves for the piece.
            for move, skip in valid_moves.items():
                if board != None:
                    temp_board = deepcopy(board)
                    temp_piece = temp_board.get_piece(piece.row, piece.col)
                    new_board = simulate_move(temp_piece, move, temp_board, game, skip)  # Simulate the move on the board.
                    moves.append(new_board)
    
    return moves

In [5]:
def alpha_beta(position, depth, alpha, beta, max_player, game, human_play):
    """
    Implementation of the Alpha-Beta Pruning algorithm over minimax
    """
    if depth == 0:
        # If depth is 0, evaluate the current position and return the score along with the position.
        return position.evaluate(game.board.player), position

    if max_player:
        if human_play == True: 
            if game.turn == "red":
                color_m = "red"  # Determine the color of the opponent's pieces
            else:
                color_m = "white"  # Determine the color of the own pieces
        else:
            if game.turn == "red":
                color_m = "white" 
            else:
                color_m = "red"  
        best_move = None
        for move in get_all_moves(position, color_m, game):
            # Recursively call alpha_beta with the next move and update alpha value.
            evaluation = alpha_beta(move, depth-1, alpha, beta, False, game, human_play)[0]
            if evaluation > alpha:
                alpha = evaluation
                best_move = move
            if alpha >= beta:
                # Perform pruning if alpha is greater than or equal to beta.
                break

        return alpha, best_move
    else:
        best_move = None
        for move in get_all_moves(position, game.turn, game):
            # Recursively call alpha_beta with the next move and update beta value.
            evaluation = alpha_beta(move, depth-1, alpha, beta, True, game, human_play)[0]
            if evaluation < beta:
                beta = evaluation
                best_move = move
            if beta <= alpha:
                # Perform pruning if beta is less than or equal to alpha.
                break

        return beta, best_move

In [6]:
#INITIALLY CHECKING THE ALGORITHM USING THIS
def minimax(position, depth, max_player, game, human_play):
    """
    Implementation of the minimax algorithm for game AI.
    """
    if depth == 0:
        return position.evaluate(game.board.player), position  # Return evaluation of current position and the position itself
    
    if max_player:
        maxEval = float('-inf')  #
        best_move = None 
        if human_play == True: 
            if game.turn == "red":
                color_m = "red"  # Determine the color of the opponent's pieces
            else:
                color_m = "white"  # Determine the color of the own pieces
        else:
            if game.turn == "red":
                color_m = "white" 
            else:
                color_m = "red"  
        for move in get_all_moves(position, color_m, game): 
            evaluation = minimax(move, depth-1, False, game, human_play)[0] 
            maxEval = max(maxEval, evaluation)  # Update the maximum evaluation score
            if maxEval == evaluation:
                best_move = move  # Update the best move if a better move is found
        
        return maxEval, best_move  
    else:
        minEval = float('inf')  
        best_move = None  
        for move in get_all_moves(position, game.turn, game):  
            evaluation = minimax(move, depth-1, True, game, human_play)[0]  
            minEval = min(minEval, evaluation)  # Update the minimum evaluation score
            if minEval == evaluation:
                best_move = move  
        
        return minEval, best_move  # Return the minimum evaluation score and the best move found

In [7]:
#HUMAN VS AI
def human_AI(play1):
    player1 = play1
    player2 = None
    obj = Game(player1, player2)
    start_time = time.time()
    counter = 0
    
    while counter < 40:
        
        old_pieces = obj.board.red_left + obj.board.white_left
        if obj.turn == "red":
            value, new_board = alpha_beta(obj.get_board(), 3, float("-inf"), float("inf"), obj.turn, obj, True)
            #new_board = iterative_deepening_alpha_beta(obj.get_board(), 4, obj, False)

            #value, new_board = minimax(obj.get_board(), 2, obj.turn, obj, True)
            obj.ai_move(new_board)
            print(new_board.board)
            gui = Checkerboard(obj.board.board)

        else:
            gui = Checkerboard(obj.board.board)
            row, col = gui.run()
            obj.select(row, col)
            
            
        print(obj.turn)
        new_pieces = obj.board.red_left + obj.board.white_left
        difference = old_pieces - new_pieces
        if difference > 0:
            counter = 0
        else:
            counter += 1
        print("DIFF: ", old_pieces - new_pieces)
        winner = obj.winner()

        if winner == "red":
            best_player = obj.player1
        else:
            best_player = obj.player2

    end_time = time.time()
    print("Duration: ", end_time - start_time)

    return new_board, winner, best_player

In [8]:
def AI_vs_AI(player_1, player_2):
    player1 = player_1
    player2 = player_2
    obj = Game(player1, player2)
    start_time = time.time()
    counter = 0
    while counter < 40:
        old_pieces = obj.board.red_left + obj.board.white_left
        # old_pieces = len(obj.board.get_all_pieces(obj.turn)) + len(obj.board.get_all_pieces(opponent))
        #new_board = iterative_deepening_alpha_beta(obj.get_board(), 4, obj, False)

        value, new_board = alpha_beta(obj.get_board(), 3, float("-inf"), float("inf"), obj.turn, obj, False)

        
        #value, new_board = minimax(obj.get_board(), 2, obj.turn, obj, False)
        #print(obj.turn)
        # print(new_board.board)
        obj.ai_move(new_board)
        new_pieces = obj.board.red_left + obj.board.white_left
        difference = old_pieces - new_pieces
        if difference > 0:
            counter = 0
        else:
            counter += 1
        # print(counter)
        # print("DIFF: ", old_pieces - new_pieces)
        winner = obj.winner()

        if winner == "white":
            best_player = obj.player1
        else:
            best_player = obj.player2

    end_time = time.time()
    #print("Duration: ", end_time - start_time)

    return new_board, winner, best_player

In [9]:
def fitnessForBinaryTournamentDania(parent_1, parent_2, chromosomes):

    parent1win = 0
    parent1loss = 0

    parent2win = 0
    parent2loss = 0

    newchromosomes = chromosomes.copy()
    #chromosomes.remove(parent_1)
    #chromosomes.remove(parent_2)

    newchromosomes.remove(parent_1)
    newchromosomes.remove(parent_2)
    
    parent1Score = 0
    parent2Score = 0

    randomPlayers = np.random.choice(newchromosomes, size=3, replace=False)
   
    for player2 in randomPlayers:
        print("p1 match")
        obj = Game(parent_1, player2)
        new_board, winner, best_player = AI_vs_AI(parent_1, player2)

        # if winner=="draw":
        #     parent1Score += 1

        #     player2.draw += 1
        #     parent_1.draw += 1

        #     parent_1.score += 1
        #     player2.score += 1

        # else:
        if winner == 'white':
            # print("bnum: ",best_player.number)
            # parent1Score -= 1
            # parent_1.loss += 1
            # player2.win += 1
            # parent_1.score -= 1
            # player2.score += 2
            parent_1.win += 1
            parent1win += 1

            player2.loss += 1

        else:
            # parent1Score += 2
            # parent_1.win += 1
            # player2.loss += 1
            # parent_1.score += 2
            # player2.score -= 1
            player2.win += 1
            parent_1.loss += 1

            parent1loss += 1
        
        player2.score = player2.win - player2.loss
        parent_1.score = parent_1.win - parent_1.loss
        #parent1Score = parent_1.score
        parent1Score = parent1win - parent1loss

    for player2 in randomPlayers:
        print("p2 match")
        obj = Game(parent_2, player2)
        new_board, winner, best_player = AI_vs_AI(parent_2, player2)

        # if winner=="draw":
        #     parent2Score += 1

        #     parent_2.draw += 1
        #     player2.draw += 1

        #     parent_2.score += 1
        #     player2.score += 1

        # else:
        if winner == 'white':
            # print("winner", winner, best_player)
            
            # print(parent_2)
            # parent2Score -= 1
            # parent_2.loss += 1
            # player2.win += 1
            # parent_2.score -= 1
            # player2.score += 2
            parent_2.win += 1
            parent2win += 1

            player2.loss += 1
        else:
            # parent2Score += 2
            # player2.loss += 1
            # parent_2.win += 1
            # parent_2.score += 2
            # player2.score -= 1
            player2.win += 1
            parent_2.loss += 1
            parent2loss += 1
        
        player2.score = player2.win - player2.loss
        parent_2.score = parent_2.win - parent_2.loss
        #parent2Score = parent_2.score
        parent2Score = parent2win - parent2loss      


    if parent2Score >= parent1Score:
        return parent_2
    else:
        return parent_1

    

In [10]:
def fitnessForBinaryTournamentDaniav(parent_1, parent_2):
    
    randomturn = random.randint(0, 1)
    
    if randomturn == 0:

        parent1score = 0
        parent2score = 0

        player1 = parent_1
        player2 = parent_2

        obj = Game(player1, player2)
        new_board, winner, best_player = AI_vs_AI(player1, player2)

        if winner == 'white':
            player1.win += 1
            player2.loss += 1

            parent1score += 1
        else:
            player2.win += 1
            player1.loss += 1

            parent2score += 1

        obj = Game(player2, player1)
        new_board, winner, best_player = AI_vs_AI(player2, player1)

        if winner == 'red':
            player1.win += 1
            player2.loss += 1

            parent1score += 1
        else:
            player2.win += 1
            player1.loss += 1

            parent2score += 1

        obj = Game(player1, player2)
        new_board, winner, best_player = AI_vs_AI(player1, player2)

        if winner == 'white':
            player1.win += 1
            player2.loss += 1

            parent1score += 1
        else:
            player2.win += 1
            player1.loss += 1

            parent2score += 1

        if parent2score > parent1score:
            return player2
        elif parent2score < parent1score:
            return player1
        else:
            r = random.randint(0, 1)

            if r == 1:
                return player1
            else:
                return player2

    else:
        parent1score = 0
        parent2score = 0

        player1 = parent_2
        player2 = parent_1

        obj = Game(player1, player2)
        new_board, winner, best_player = AI_vs_AI(player1, player2)

        if winner == 'white':
            player1.win += 1
            player2.loss += 1

            parent2score += 1
        else:
            player2.win += 1
            player1.loss += 1

            parent1score += 1

        obj = Game(player2, player1)
        new_board, winner, best_player = AI_vs_AI(player2, player1)

        if winner == 'red':
            player1.win += 1
            player2.loss += 1

            parent2score += 1
        else:
            player2.win += 1
            player1.loss += 1

            parent1score += 1

        obj = Game(player1, player2)
        new_board, winner, best_player = AI_vs_AI(player1, player2)

        if winner == 'white':
            player1.win += 1
            player2.loss += 1

            parent2score += 1
        else:
            player2.win += 1
            player1.loss += 1

            parent1score += 1

        if parent2score > parent1score:
            return player1
        elif parent2score < parent1score:
            return player2
        else:
            r = random.randint(0, 1)

            if r == 1:
                return player1
            else:
                return player2


In [11]:
def fitnessForBinaryTournament(parent_1, parent_2, chromosomes):

    newchromosomes = chromosomes.copy()
    #chromosomes.remove(parent_1)
    #chromosomes.remove(parent_2)

    newchromosomes.remove(parent_1)
    newchromosomes.remove(parent_2)
    
    parent1Score = 0
    parent2Score = 0

    randomPlayers = np.random.choice(newchromosomes, size=3, replace=False)
   
    for player2 in randomPlayers:
        
        obj = Game(parent_1, player2)
        new_board, winner, best_player = AI_vs_AI(parent_1, player2)

        if winner=="draw": 
            player2.draw += 1
            parent_1.draw += 1

        else:
            if winner == 'red':

                parent1Score -= 1

                parent_1.loss += 1
                player2.win += 1

                parent_1.score -= 1
                player2.score += 2

            else:

                parent1Score += 2

                parent_1.win += 1
                player2.loss += 1

                parent_1.score += 2
                player2.score -= 1

    for player2 in randomPlayers:
        obj = Game(parent_2, player2)
        new_board, winner, best_player = AI_vs_AI(parent_2, player2)

        if winner=="draw":
            
            parent_2.draw += 1
            player2.draw += 1

        else:
            if winner == 'red':
               
                parent2Score -= 1

                parent_2.loss += 1
                player2.win += 1

                parent_2.score -= 1
                player2.score += 3

            else:
                parent2Score += 3

                player2.loss += 1
                parent_2.win += 1

                parent_2.score += 3
                player2.score -= 1
                


    if parent2Score >= parent1Score:
        return parent_2
    else:
        return parent_1

In [12]:
def parentSelection(chromosomes, type):

    if type == "binary_tournament":

        nchromosomes = chromosomes.copy()

        index = np.random.choice(nchromosomes, 2, replace=False) 

        parent_1 = index[0]
        parent_2 = index[1]
       
        # Choose the fittest chromosome as the first parent
        
        parent1 = fitnessForBinaryTournamentDaniav(parent_1, parent_2)
        
        nchromosomes.remove(parent1)
        
        index = np.random.choice(nchromosomes, 2, replace=False) 
        # repeats for second parent
        parent_1 = index[0]
        parent_2 = index[1]
        
        parent2 = fitnessForBinaryTournamentDaniav(parent_1, parent_2)

        nchromosomes.remove(parent2)

        index = np.random.choice(nchromosomes, 2, replace=False) 
        # repeats for second parent
        parent_1 = index[0]
        parent_2 = index[1]
        
        parent3 = fitnessForBinaryTournamentDaniav(parent_1, parent_2)

        nchromosomes.remove(parent3)
        
        index = np.random.choice(nchromosomes, 2, replace=False) 
        # repeats for second parent
        parent_1 = index[0]
        parent_2 = index[1]
        
        parent4 = fitnessForBinaryTournamentDaniav(parent_1, parent_2)
        


    return parent1, parent2, parent3, parent4

In [13]:
def crossover(parent1, parent2):

    p1first_layer_weights, p1first_layer_bias, p1second_layer_weights, p1second_layer_bias, p1third_layer_weights,  p1third_layer_bias = parent1.getWeights()
    p2first_layer_weights, p2first_layer_bias, p2second_layer_weights, p2second_layer_bias, p2third_layer_weights,  p2third_layer_bias = parent2.getWeights()
    
    crossoverPoint1 = random.randint(0, len(p1first_layer_weights) - 1)

    o1first_layer_weights = []
    o2first_layer_weights = []

    for i in range(len(p1first_layer_weights)):
        
        o1 = np.concatenate((p1first_layer_weights[i][:crossoverPoint1],(p2first_layer_weights[i][crossoverPoint1:])))
        o2 = np.concatenate((p2first_layer_weights[i][:crossoverPoint1], p1first_layer_weights[i][crossoverPoint1:]))

        o1first_layer_weights.append(o1)
        o2first_layer_weights.append(o2)

    crossoverPoint2 = random.randint(0, len(p1first_layer_bias) - 1)
   
    o1first_layer_bias = []
    o2first_layer_bias = []

    for i in range(len(p1first_layer_bias)):

        o1 = np.concatenate((p1first_layer_bias[i][:crossoverPoint2], p2first_layer_bias[i][crossoverPoint2:]))
        o2 = np.concatenate((p2first_layer_bias[i][:crossoverPoint2], p1first_layer_bias[i][crossoverPoint2:]))

        o1first_layer_bias.append(o1)
        o2first_layer_bias.append(o2)

    crossoverPoint3 = random.randint(0, len(p1second_layer_weights) - 1)
   
    o1second_layer_weights = []
    o2second_layer_weights = []

    for i in range(len(p1second_layer_weights)):

        o1 = np.concatenate((p1second_layer_weights[i][:crossoverPoint3], p2second_layer_weights[i][crossoverPoint3:]))
        o2 = np.concatenate((p2second_layer_weights[i][:crossoverPoint3], p1second_layer_weights[i][crossoverPoint3:]))

        o1second_layer_weights.append(o1)
        o2second_layer_weights.append(o2)

    crossoverPoint4 = random.randint(0, len(p1second_layer_bias) - 1)

    o1second_layer_bias = []
    o2second_layer_bias = []

    for i in range(len(p1second_layer_bias)):

        o1 = np.concatenate((p1second_layer_bias[i][:crossoverPoint4], p2second_layer_bias[i][crossoverPoint4:]))
        o2 = np.concatenate((p2second_layer_bias[i][:crossoverPoint4], p1second_layer_bias[i][crossoverPoint4:]))

        o1second_layer_bias.append(o1)
        o2second_layer_bias.append(o2)

    crossoverPoint5 = random.randint(0, len(p1third_layer_weights) - 1)

    o1third_layer_weights = []
    o2third_layer_weights = []

    for i in range(len(p1third_layer_weights)):

        o1 = np.concatenate((p1third_layer_weights[i][:crossoverPoint5], p2third_layer_weights[i][crossoverPoint5:]))
        o2 = np.concatenate((p2third_layer_weights[i][:crossoverPoint5], p1third_layer_weights[i][crossoverPoint5:]))

        o1third_layer_weights.append(o1)
        o2third_layer_weights.append(o2)

    crossoverPoint6 = random.randint(0, len(p1third_layer_bias) - 1)

    o1third_layer_bias = []
    o2third_layer_bias = []

    for i in range(len(p1third_layer_bias)):

        o1 = np.concatenate((p1third_layer_bias[0][:crossoverPoint6], p2third_layer_bias[0][crossoverPoint6:]))
        o2 = np.concatenate((p2third_layer_bias[0][:crossoverPoint6], p1third_layer_bias[0][crossoverPoint6:]))

        o1third_layer_bias.append(o1)
        o2third_layer_bias.append(o2)

    offspring1 = []
    offspring1.append(o1first_layer_bias)
    offspring1.append(o1second_layer_bias)
    offspring1.append(o1third_layer_bias)

    offspring1.append(o1first_layer_weights)
    offspring1.append(o1second_layer_weights)
    offspring1.append(o1third_layer_weights)

    offspring2 = []
    offspring2.append(o2first_layer_bias)
    offspring2.append(o2second_layer_bias)
    offspring2.append(o2third_layer_bias)

    offspring2.append(o2first_layer_weights)
    offspring2.append(o2second_layer_weights)
    offspring2.append(o2third_layer_weights)

    return offspring1, offspring2

In [14]:
def uniform_crossover(parent1, parent2):

    p1_weights_biases = parent1.getWeights()
    p2_weights_biases = parent2.getWeights()

    o1_weights_biases = []
    o2_weights_biases = []

    for p1_layer, p2_layer in zip(p1_weights_biases, p2_weights_biases):
        mask = np.random.randint(0, 2, p1_layer.shape).astype(bool)
        o1_layer = np.where(mask, p1_layer, p2_layer)
        o2_layer = np.where(mask, p2_layer, p1_layer)

        o1_weights_biases.append(o1_layer)
        o2_weights_biases.append(o2_layer)

    #offspring1 = o1_weights_biases
    #offspring2 = o2_weights_biases

    p1first_layer_weights, p1first_layer_bias, p1second_layer_weights, p1second_layer_bias, p1third_layer_weights,  p1third_layer_bias = o1_weights_biases
    p2first_layer_weights, p2first_layer_bias, p2second_layer_weights, p2second_layer_bias, p2third_layer_weights,  p2third_layer_bias = o2_weights_biases
    
    offspring1 = []
    offspring1.append(p1first_layer_bias)
    offspring1.append(p1second_layer_bias)
    offspring1.append(p1third_layer_bias)

    offspring1.append(p1first_layer_weights)
    offspring1.append(p1second_layer_weights)
    offspring1.append(p1third_layer_weights)

    offspring2 = []
    offspring2.append(p2first_layer_bias)
    offspring2.append(p2second_layer_bias)
    offspring2.append(p2third_layer_bias)

    offspring2.append(p2first_layer_weights)
    offspring2.append(p2second_layer_weights)
    offspring2.append(p2third_layer_weights)

    return offspring1, offspring2


In [15]:
def mutationNew(offspring1, offspring2, mutationRate):

    o1first_layer_bias, o1second_layer_bias, o1third_layer_bias, o1first_layer_weights, o1second_layer_weights, o1third_layer_weights = offspring1
    o2first_layer_bias, o2second_layer_bias, o2third_layer_bias, o2first_layer_weights, o2second_layer_weights, o2third_layer_weights = offspring2
    
    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1first_layer_bias[0]) - 1, size=6)

        for i in range(6):
             o1first_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)

        #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2first_layer_bias[0]) - 1, size=6)


        for i in range(6):
            o2first_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)
            
        #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1second_layer_bias[0]) - 1, size=4)

        for i in range(4):
             o1second_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)

        #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2second_layer_bias[0]) - 1, size=4)


        for i in range(4):
            o2second_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)
            
        #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1third_layer_bias[0]) - 1, size=2)

        for i in range(2):
             o1third_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)

        #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2third_layer_bias[0]) - 1, size=2)


        for i in range(2):
            o2third_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)
            
        #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    for i in range(len(o1first_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1first_layer_weights[0]) - 1, size=6)

            for j in range(6):
                o1first_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)

            #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2first_layer_weights[0]) - 1, size=6)


            for j in range(6):
                o2first_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)
                
            #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)


    for i in range(len(o1second_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1second_layer_weights[0]) - 1, size=4)

            for j in range(4):
                o1second_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)

            #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2second_layer_weights[0]) - 1, size=4)

            for j in range(4):
                o2second_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)
                
            #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    for i in range(len(o1third_layer_weights) - 1):
         if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1third_layer_weights[0]) - 1, size=2)

            for j in range(2):
                o1third_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)

            #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2third_layer_weights[0]) - 1, size=2)

            for j in range(2):
                o2third_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)
                
            #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)
    mutatedOffspring1 = []
    mutatedOffspring1.append(o1first_layer_bias)
    mutatedOffspring1.append(o1second_layer_bias)
    mutatedOffspring1.append(o1third_layer_bias)

    mutatedOffspring1.append(o1first_layer_weights)
    mutatedOffspring1.append(o1second_layer_weights)
    mutatedOffspring1.append(o1third_layer_weights)

    mutatedOffspring2 = []
    mutatedOffspring2.append(o2first_layer_bias)
    mutatedOffspring2.append(o2second_layer_bias)
    mutatedOffspring2.append(o2third_layer_bias)

    mutatedOffspring2.append(o2first_layer_weights)
    mutatedOffspring2.append(o2second_layer_weights)
    mutatedOffspring2.append(o2third_layer_weights)

    return mutatedOffspring1, mutatedOffspring2

In [16]:
def mutation(offspring1, offspring2, mutationRate):

    o1first_layer_bias, o1second_layer_bias, o1third_layer_bias, o1first_layer_weights, o1second_layer_weights, o1third_layer_weights = offspring1
    o2first_layer_bias, o2second_layer_bias, o2third_layer_bias, o2first_layer_weights, o2second_layer_weights, o2third_layer_weights = offspring2
    
    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1first_layer_bias[0]) - 1, size=2)
        o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)
        

        rand_nums = np.random.randint(0, len(o2first_layer_bias[0]) - 1, size=2)
        o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1second_layer_bias[0]) - 1, size=2)
        o1second_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o1second_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)
        
        rand_nums = np.random.randint(0, len(o2second_layer_bias[0]) - 1, size=2)
        o2second_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o2second_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1third_layer_bias[0]) - 1, size=2)
        o1third_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o1third_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2third_layer_bias[0]) - 1, size=2)
        o2third_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o2third_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    for i in range(len(o1first_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1first_layer_weights[0]) - 1, size=2)
            o1first_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o1first_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2first_layer_weights[0]) - 1, size=2)
            o2first_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o2first_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)


    for i in range(len(o1second_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1second_layer_weights[0]) - 1, size=2)
            o1second_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o1second_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2second_layer_weights[0]) - 1, size=2)
            o2second_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o2second_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)


    for i in range(len(o1third_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1third_layer_weights[0]) - 1, size=2)
            o1third_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o1third_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2third_layer_weights[0]) - 1, size=2)
            o2third_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o2third_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

    mutatedOffspring1 = []
    mutatedOffspring1.append(o1first_layer_bias)
    mutatedOffspring1.append(o1second_layer_bias)
    mutatedOffspring1.append(o1third_layer_bias)

    mutatedOffspring1.append(o1first_layer_weights)
    mutatedOffspring1.append(o1second_layer_weights)
    mutatedOffspring1.append(o1third_layer_weights)

    mutatedOffspring2 = []
    mutatedOffspring2.append(o2first_layer_bias)
    mutatedOffspring2.append(o2second_layer_bias)
    mutatedOffspring2.append(o2third_layer_bias)

    mutatedOffspring2.append(o2first_layer_weights)
    mutatedOffspring2.append(o2second_layer_weights)
    mutatedOffspring2.append(o2third_layer_weights)

    return mutatedOffspring1, mutatedOffspring2

In [17]:
def calculateFitness(population):

    for i in range(len(population) - 1):

        player1 = population[i]

        challengers = population[:i] +  population[i+1:]

        randomPlayers = np.random.choice(challengers, size=5, replace=False)

        for player2 in randomPlayers:
            obj = Game(player1, player2)
            new_board, winner, best_player = AI_vs_AI(player1, player2)

            if winner=="draw":
                player1.score +=1
                player2.score +=1
            else:
                best_player.score +=2

                if winner == 'red':
                    player1.score -= 1
                else:
                    player2.score -= 1

            #print("game complete")

In [18]:
def calculateFitnessOffspring(offspring1, offspring2, population):
    
    players = [offspring1, offspring2]

    for player1 in players:

        randomPlayers = np.random.choice(population, size=5, replace=False)
    
        for player2 in randomPlayers:
            obj = Game(offspring1, player2)
            new_board, winner, best_player = AI_vs_AI(offspring1, player2)

            if winner=="draw":
                offspring1.score +=1
                player2.score +=1
            else:
                best_player.score +=2

                if winner == 'red':
                    player2.score -= 1
                else:
                    offspring1.score -= 1

                
    return population + players


    

In [19]:
def fitnessForBinaryTournament_2Dania(parent_1, parent_2, chromosomes):
    
    newchromosomes = chromosomes.copy()

    #newchromosomes.remove(parent_1)
    #newchromosomes.remove(parent_2)

    parent1Score = 0
    parent2Score = 0
    
    parent1win = 0
    parent2win = 0

    parent1loss = 0
    parent2loss = 0

   
    randomPlayers = np.random.choice(newchromosomes, size=3, replace=False)
    
    for player2 in randomPlayers:
        print("p1 match")
        obj = Game(parent_1, player2)
        new_board, winner, best_player = AI_vs_AI(parent_1, player2)

        # if winner=="draw":
        #     parent1Score += 1

        #     player2.draw += 1
        #     parent_1.draw += 1

        #     parent_1.score += 1
        #     player2.score += 1

        # else:
        if winner == 'white':
            # print("bnum: ",best_player.number)
            # parent1Score -= 1
            # parent_1.loss += 1
            # player2.win += 1
            # parent_1.score -= 1
            # player2.score += 2
            parent_1.win += 1
            player2.loss += 1

            parent1win += 1

        else:
            # parent1Score += 2
            # parent_1.win += 1
            # player2.loss += 1
            # parent_1.score += 2
            # player2.score -= 1
            player2.win += 1
            parent_1.loss += 1

            parent1loss += 1
        
        player2.score = player2.win - player2.loss
        parent_1.score = parent_1.win - parent_1.loss
        #parent1Score = parent_1.score
        parent1Score = parent1win - parent1loss

    for player2 in randomPlayers:

        print("p2 match")
        obj = Game(parent_2, player2)
        new_board, winner, best_player = AI_vs_AI(parent_2, player2)

        # if winner=="draw":
        #     parent2Score += 1

        #     parent_2.draw += 1
        #     player2.draw += 1

        #     parent_2.score += 1
        #     player2.score += 1

        # else:
        if winner == 'white':
            # print("winner", winner, best_player)
            
            # print(parent_2)
            # parent2Score -= 1
            # parent_2.loss += 1
            # player2.win += 1
            # parent_2.score -= 1
            # player2.score += 2
            parent_2.win += 1
            player2.loss += 1

            parent2win += 1

        else:
            # parent2Score += 2
            # player2.loss += 1
            # parent_2.win += 1
            # parent_2.score += 2
            # player2.score -= 1
            player2.win += 1
            parent_2.loss += 1

            parent2loss += 1
        
        player2.score = player2.win - player2.loss
        parent_2.score = parent_2.win - parent_2.loss
        #parent2Score = parent_2.score
            
        parent2Score = parent2win - parent2loss


    if parent2Score > parent1Score:
        return parent_2
    else:
        return parent_1

    

In [20]:
def fitnessForBinaryTournament_v2Dania(parent_1, parent_2, chromosomes):
    
    randomturn = random.randint(0, 1)
    
    if randomturn == 0:

        parent1score = 0
        parent2score = 0

        player1 = parent_1
        player2 = parent_2

        obj = Game(player1, player2)
        new_board, winner, best_player = AI_vs_AI(player1, player2)

        if winner == 'white':
            player1.win += 1
            player2.loss += 1

            parent1score += 1
        else:
            player2.win += 1
            player1.loss += 1

            parent2score += 1

        obj = Game(player2, player1)
        new_board, winner, best_player = AI_vs_AI(player2, player1)

        if winner == 'red':
            player1.win += 1
            player2.loss += 1

            parent1score += 1
        else:
            player2.win += 1
            player1.loss += 1

            parent2score += 1

        obj = Game(player1, player2)
        new_board, winner, best_player = AI_vs_AI(player1, player2)

        if winner == 'white':
            player1.win += 1
            player2.loss += 1

            parent1score += 1
        else:
            player2.win += 1
            player1.loss += 1

            parent2score += 1

        if parent2score > parent1score:
            return player2
        elif parent2score < parent1score:
            return player1
        else:
            r = random.randint(0, 1)

            if r == 1:
                return player1
            else:
                return player2

    else:
        parent1score = 0
        parent2score = 0

        player1 = parent_2
        player2 = parent_1

        obj = Game(player1, player2)
        new_board, winner, best_player = AI_vs_AI(player1, player2)

        if winner == 'white':
            player1.win += 1
            player2.loss += 1

            parent2score += 1
        else:
            player2.win += 1
            player1.loss += 1

            parent1score += 1

        obj = Game(player2, player1)
        new_board, winner, best_player = AI_vs_AI(player2, player1)

        if winner == 'red':
            player1.win += 1
            player2.loss += 1

            parent2score += 1
        else:
            player2.win += 1
            player1.loss += 1

            parent1score += 1

        obj = Game(player1, player2)
        new_board, winner, best_player = AI_vs_AI(player1, player2)

        if winner == 'white':
            player1.win += 1
            player2.loss += 1

            parent2score += 1
        else:
            player2.win += 1
            player1.loss += 1

            parent1score += 1

        if parent2score > parent1score:
            return player1
        elif parent2score < parent1score:
            return player2
        else:
            r = random.randint(0, 1)

            if r == 1:
                return player1
            else:
                return player2


In [21]:
def fitnessForBinaryTournament_2(parent_1, parent_2, chromosomes):
    
    newchromosomes = chromosomes.copy()

    #newchromosomes.remove(parent_1)
    #newchromosomes.remove(parent_2)

    parent1Score = 0
    parent2Score = 0
   
    randomPlayers = np.random.choice(newchromosomes, size=3, replace=False)
    
    for player2 in randomPlayers:
      
        obj = Game(parent_1, player2)
        new_board, winner, best_player = AI_vs_AI(parent_1, player2)

        if winner=="draw":
            player2.draw += 1
            parent_1.draw += 1

        else:
            if winner == 'red':
                parent1Score -= 1

                player2.win += 1
                parent_1.loss += 1

                parent_1.score -= 1
                player2.score += 3
            else:
                parent1Score += 3

                parent_1.win += 1
                player2.loss += 1

                parent_1.score += 3
                player2.score -= 1

    for player2 in randomPlayers:
       
        obj = Game(parent_2, player2)
        new_board, winner, best_player = AI_vs_AI(parent_2, player2)

        if winner=="draw":

            parent_2.draw += 1
            player2.draw += 1

        else:
            if winner == 'red':
                parent2Score -= 1
                
                parent_2.loss += 1
                player2.win += 1

                parent_2.score -= 1
                player2.score += 3

            else:
                parent2Score += 3

                parent_2.win += 1
                player2.loss += 1

                parent_2.score += 3
                player2.score -= 1

    if parent2Score >= parent1Score:
        return parent_2
    else:
        return parent_1

In [22]:
def survivorSelection(newPopulation, type, totalnum):

    finalPopulation = []
    
    if type == "tournament":
       
       nnewPopulation = newPopulation.copy()

       for i in range(totalnum):
           
           index = np.random.choice(nnewPopulation, 2, replace=False) 

           parent1 = index[0]
           parent2 = index[1]
           
           #parent1 = random.choice(newPopulation)
           #parent2 = random.choice(newPopulation)

           chosen = fitnessForBinaryTournament_v2Dania(parent1, parent2, nnewPopulation)
           
           finalPopulation.append(chosen)
           nnewPopulation.remove(chosen)
    
    return finalPopulation

In [23]:
def DaniasCrossover(parent1, parent2):
    p1first_layer_weights, p1first_layer_bias, p1second_layer_weights, p1second_layer_bias, p1third_layer_weights,  p1third_layer_bias = parent1.getWeights()
    p2first_layer_weights, p2first_layer_bias, p2second_layer_weights, p2second_layer_bias, p2third_layer_weights,  p2third_layer_bias = parent2.getWeights()
    
    random1 = random.random()

    o1first_layer_weights = []
    o2first_layer_weights = []

    for i in range(len(p1first_layer_weights)):
        o1 = 0.5 * (((1 + random1) * p1first_layer_weights[i]) + ((1- random1) * p2first_layer_weights[i]))
        o2 = 0.5 * (((1 - random1) * p1first_layer_weights[i]) + ((1 + random1) * p2first_layer_weights[i]))

        o1first_layer_weights.append(o1)
        o2first_layer_weights.append(o2)

    random2 = random.random()
   
    o1first_layer_bias = []
    o2first_layer_bias = []

    for i in range(len(p1first_layer_bias)):

        o1 = 0.5 * (((1 + random2) * p1first_layer_bias[i]) + ((1- random2) * p2first_layer_bias[i]))
        o2 = 0.5 * (((1 - random2) * p1first_layer_bias[i]) + ((1 + random2) * p2first_layer_bias[i]))

        o1first_layer_bias.append(o1)
        o2first_layer_bias.append(o2)

    random3 = random.random()
   
    o1second_layer_weights = []
    o2second_layer_weights = []

    for i in range(len(p1second_layer_weights)):

        o1 = 0.5 * (((1 + random3) * p1second_layer_weights[i]) + ((1- random3) * p2second_layer_weights[i]))
        o2 = 0.5 * (((1 - random3) * p1second_layer_weights[i]) + ((1 + random3) * p2second_layer_weights[i]))

        o1second_layer_weights.append(o1)
        o2second_layer_weights.append(o2)

    random4 = random.random()

    o1second_layer_bias = []
    o2second_layer_bias = []
    
    for i in range(len(p2second_layer_bias)):
        o1 = 0.5 * (((1 + random4) * p1second_layer_bias[i]) + ((1- random4) * p2second_layer_bias[i]))
        o2 = 0.5 * (((1 - random4) * p1second_layer_bias[i]) + ((1 + random4) * p2second_layer_bias[i]))

        o1second_layer_bias.append(o1)
        o2second_layer_bias.append(o2)

    random5 = random.random()
   
    o1third_layer_weights = []
    o2third_layer_weights = []

    for i in range(len(p1third_layer_weights)):

        o1 = 0.5 * (((1 + random5) * p1third_layer_weights[i]) + ((1- random5) * p2third_layer_weights[i]))
        o2 = 0.5 * (((1 - random5) * p1third_layer_weights[i]) + ((1 + random5) * p2third_layer_weights[i]))

        o1third_layer_weights.append(o1)
        o2third_layer_weights.append(o2)

    random6 = random.random()

    o1third_layer_bias = []
    o2third_layer_bias = []

    for i in range(len(p1third_layer_bias)):

        o1 = 0.5 * (((1 + random6) * p1third_layer_bias[i]) + ((1- random6) * p2third_layer_bias[i]))
        o2 = 0.5 * (((1 - random6) * p1third_layer_bias[i]) + ((1 + random6) * p2third_layer_bias[i]))

        o1third_layer_bias.append(o1)
        o2third_layer_bias.append(o2)

    offspring1 = []
    offspring1.append(o1first_layer_bias)
    offspring1.append(o1second_layer_bias)
    offspring1.append(o1third_layer_bias)

    offspring1.append(o1first_layer_weights)
    offspring1.append(o1second_layer_weights)
    offspring1.append(o1third_layer_weights)

    offspring2 = []
    offspring2.append(o2first_layer_bias)
    offspring2.append(o2second_layer_bias)
    offspring2.append(o2third_layer_bias)

    

    offspring2.append(o2first_layer_weights)
    offspring2.append(o2second_layer_weights)
    offspring2.append(o2third_layer_weights)

    return offspring1, offspring2

In [24]:
def NueroEvolution2(population, generations):

    best_overall_player_score = 0

    chromosomes = []

    #Initialise random population
    for i in range(population):
        chromosomes.append(evolutionary_player(i))
    
    count = 25
    
    for n in range(generations):
        
        print("Generation: ", n)

        parent1, parent2, parent3, parent4 = parentSelection(chromosomes, "binary_tournament")

        offspring1, offspring2 = crossover(parent1, parent2)  
        offspring3, offspring4 = crossover(parent3, parent4)
        print("Crossover complete")

        offspring1AfterMutation, offspring2AfterMutation = mutationNew(offspring1, offspring2, mutationRate=0.2)
        offspring3AfterMutation, offspring4AfterMutation = mutationNew(offspring3, offspring4, mutationRate=0.2)

        parent1player, parent2player = createNeuralNetwork(offspring1AfterMutation, offspring2AfterMutation, count)
        count += 2

        parent3player, parent4player = createNeuralNetwork(offspring3AfterMutation, offspring4AfterMutation, count)

        print("Mutation complete")

        newPopulation = chromosomes + [parent1player, parent2player, parent3player, parent4player]
        
        chromosomes = survivorSelection(newPopulation, "tournament", population)

        
        print("Generation Scores")
        for i in chromosomes:
            
            print("wins:", i.win, "losses:", i.loss, "draws:", i.draw, "score", i.score)
        
    

        count += 2

    
    return chromosomes
    #max_fit = 0
    #player = None
    #for i in chromosomes:
    #    print(i)
    #    if i.score > max_fit:
    #        max_fit = i.score
    #        player = i
    #return player

    

In [25]:
allchromosomes = NueroEvolution2(12, 5)

Generation:  0
Crossover complete
Mutation complete
Generation Scores
wins: 2 losses: 1 draws: 0 score 0
wins: 2 losses: 1 draws: 0 score 0
wins: 3 losses: 6 draws: 0 score 0
wins: 2 losses: 1 draws: 0 score 0
wins: 5 losses: 1 draws: 0 score 0
wins: 7 losses: 5 draws: 0 score 0
wins: 3 losses: 6 draws: 0 score 0
wins: 3 losses: 6 draws: 0 score 0
wins: 2 losses: 1 draws: 0 score 0
wins: 5 losses: 1 draws: 0 score 0
wins: 6 losses: 0 draws: 0 score 0
wins: 4 losses: 2 draws: 0 score 0
Generation:  1
Crossover complete
Mutation complete
Generation Scores
wins: 7 losses: 2 draws: 0 score 0
wins: 2 losses: 4 draws: 0 score 0
wins: 9 losses: 6 draws: 0 score 0
wins: 7 losses: 11 draws: 0 score 0
wins: 12 losses: 6 draws: 0 score 0
wins: 6 losses: 6 draws: 0 score 0
wins: 6 losses: 3 draws: 0 score 0
wins: 8 losses: 1 draws: 0 score 0
wins: 3 losses: 0 draws: 0 score 0
wins: 5 losses: 7 draws: 0 score 0
wins: 8 losses: 4 draws: 0 score 0
wins: 2 losses: 4 draws: 0 score 0
Generation:  2
Cro

In [26]:
import sys
Sample_array_2 = np.arange(100)
np.set_printoptions(threshold=sys.maxsize)
#print(player2.first_layer_weights)

In [27]:
#print(player2.second_layer_weights)

In [28]:
#print(player2.third_layer_weights)

In [29]:
#print(player2.first_layer_bias)

In [30]:
#print(player2.second_layer_bias)

In [31]:
#print(player2.third_layer_bias)

In [32]:
for i in allchromosomes:
    total = i.win + i.loss + i.draw
    print("wins:", i.win, " losses:", i.loss, " draws:", i.draw, "score", i.score, " total Games", i.win + i.loss + i.draw)
    
    print(i.win/total)
   
len(allchromosomes)

wins: 15  losses: 12  draws: 0 score 0  total Games 27
0.5555555555555556
wins: 10  losses: 5  draws: 0 score 0  total Games 15
0.6666666666666666
wins: 2  losses: 1  draws: 0 score 0  total Games 3
0.6666666666666666
wins: 15  losses: 21  draws: 0 score 0  total Games 36
0.4166666666666667
wins: 2  losses: 1  draws: 0 score 0  total Games 3
0.6666666666666666
wins: 21  losses: 6  draws: 0 score 0  total Games 27
0.7777777777777778
wins: 3  losses: 0  draws: 0 score 0  total Games 3
1.0
wins: 15  losses: 15  draws: 0 score 0  total Games 30
0.5
wins: 7  losses: 2  draws: 0 score 0  total Games 9
0.7777777777777778
wins: 9  losses: 3  draws: 0 score 0  total Games 12
0.75
wins: 7  losses: 2  draws: 0 score 0  total Games 9
0.7777777777777778
wins: 3  losses: 6  draws: 0 score 0  total Games 9
0.3333333333333333


12

In [33]:
player2 = allchromosomes[5]

In [34]:
new_board, winner, best_player = human_AI(player2)
#new_board, winner, best_player = AI_vs_AI()

for i in new_board.board:
    print(i)
for i in new_board.board:
    for piece in i:
        if piece !=0:
            print("PIECE COLOR: ", piece.color, " IS KING?: ", piece.king)
print("WINNER COLOR:")
print(winner)
print("WINNER PLAYER NUM:")
print(best_player.number)
print("best first layer weights:")
print(best_player.first_layer_weights)
print("best first layer bias:")
print(best_player.first_layer_bias)
print("best second layer weights:")
print(best_player.second_layer_weights)
print("best second layer bias:")
print(best_player.second_layer_bias)
print("best third layer weights:")
print(best_player.third_layer_weights)
print("best third layer bias:")

[[0, white, 0, white, 0, white, 0, white], [white, 0, white, 0, white, 0, white, 0], [0, white, 0, white, 0, white, 0, white], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, red], [red, 0, red, 0, red, 0, 0, 0], [0, red, 0, red, 0, red, 0, red], [red, 0, red, 0, red, 0, red, 0]]
white
DIFF:  0
you have selected the piece at (2,5)
white
DIFF:  0
you have selected the piece at (3,6)
red
DIFF:  0
[[0, white, 0, white, 0, white, 0, white], [white, 0, white, 0, white, 0, white, 0], [0, white, 0, white, 0, red, 0, white], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [red, 0, red, 0, red, 0, 0, 0], [0, red, 0, red, 0, red, 0, red], [red, 0, red, 0, red, 0, red, 0]]
white
DIFF:  1
you have selected the piece at (1,6)
white
DIFF:  0
you have selected the piece at (3,4)
red
DIFF:  1
[[0, white, 0, white, 0, white, 0, white], [white, 0, white, 0, white, 0, 0, 0], [0, white, 0, white, 0, 0, 0, white], [0, 0, 0, 0, white, 0, 0, 0], [0, 0, 0, red, 0, 0, 0, 0], [red, 0, 0, 0, red, 0, 0, 0], [

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
new_board, winner, best_player = human_AI(player2)
#new_board, winner, best_player = AI_vs_AI()

for i in new_board.board:
    print(i)
for i in new_board.board:
    for piece in i:
        if piece !=0:
            print("PIECE COLOR: ", piece.color, " IS KING?: ", piece.king)
print("WINNER COLOR:")
print(winner)
print("WINNER PLAYER NUM:")
print(best_player.number)
print("best first layer weights:")
print(best_player.first_layer_weights)
print("best first layer bias:")
print(best_player.first_layer_bias)
print("best second layer weights:")
print(best_player.second_layer_weights)
print("best second layer bias:")
print(best_player.second_layer_bias)
print("best third layer weights:")
print(best_player.third_layer_weights)
print("best third layer bias:")